# Welcome to *The Christmas Card Conundrum*! #

Each year Kaggle hosts a special Santa-themed optimization challenge. This year you're challenged to optimize the path of a robotic arm over the points of an image, minimizing both the movement of the arm and the change in color from step to step.

In this notebook we will learn how to break down submission cost into individual components to see where most of the potential optimization is possible. To do so we will approximate optimal solution from below as good as we can.

# So what the step cost consists of?

It may appear from the competition description that cost of individual movement of an arm consists of 2 parts: cost of color change and cost of arm legs rotation.

However it can be useful to further split cost of arm movement into 2 components: geometrical and configurational. Lets talk about what it means, why and how it is possible first.

# Geometrical vs configurational cost

First important consideration is that rotation of one leg always translates into movement of arm tip to one of 4 adjacent cells of the image. It means that combination of rotations of 2 legs moves arm tip by 0 or 2 units if we add up x and y distance between source and destination cells. Similarly when rotating 3 legs we move arm tip by 1 or 3 units. 4 legs - by 0, 2 or 4 units, etc.

It makes sense to define distance between cells of an image as sum of differences in their x and y coordinates. In math this measure divided by number of dimentions is called $L_1 norm$. Just like regular Eucledian distance is called $L_2 norm$ because it uses squares of distances by each coordinate.

As we remember that usually there are multiple changes to arm configuration that move arm tip to the same cell of the image, any configuration change that moves arm tip to lower distance then number of legs that were rotated is suboptimal and can be optimized by removing 2 or more leg rotations that move arm tip in the opposite directions. Thus the is a natural cost of the step induced by geometrical distance between starting and ending points, along with possible additional cost from rotating spare arm legs.

To reiterate, geometrical cost comes from the movement of arm tip and in some sense is inevitable.

Configurational cost comes from spare leg rotations that are not necessary for arm tip movement.

# So how do we split arm movement cost of entire solution?

There is a tremendous number of ways to draw single color picture of size 257x257 (e.g. "black square") in exactly 257x257=66049 moves each involving single leg rotation, thus not inducing any additional score. So any step where arm tip is moved by more then one unit can be considered geometrically not optimal. And any time we paint over a point which was already painted, we introduce whole spare step, which is one of the worst types of waste of score possible.

Now we can split total cost of arm movement into 4 components:

cost of optimal colorless solution (66049) + number of spare steps + additional cost of movements by more then one unit + cost of spare leg rotations.

# What about color cost?

Costs of color changes also do not come for free. Completely or partially ignoring geometrical limitations of the problem, we can find lower bound on sum of all color transitions, thus finding closer lower bound of optimal solution, then just from colorless score.

E.g. we know that changing configuration of 8 legs cannot move arm tip more then by 8 units in our $L_1$ distance. For each cell in our image we can find 2 closest colors withing 8 unit distance by color transition score. We can also take into account that choosing further point comes with a price of $sqrt(distance)-1$. Adding up scores of all these transitions divided by 2 will give some lower bound on color-induced score.

Let's get to coding.
Below is some useful code from [Getting Started](https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2022) Notebook, which I forked from.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
from functools import *
from itertools import *
from pathlib import Path
from PIL import Image

plt.style.use('seaborn-whitegrid')

data_dir = Path('/kaggle/input/santa-2022')
df_image = pd.read_csv(data_dir / 'image.csv')

Here we'll define some convenience functions to transform the image between various formats and then transform the dataframe into a numpy array.

In [ ]:
# Functions to map between cartesian coordinates and array indexes
def cartesian_to_array(x, y, shape):
    m, n = shape[:2]
    i = (n - 1) // 2 - y
    j = (n - 1) // 2 + x
    if i < 0 or i >= m or j < 0 or j >= n:
        raise ValueError("Coordinates not within given dimensions.")
    return i, j


def array_to_cartesian(i, j, shape):
    m, n = shape[:2]
    if i < 0 or i >= m or j < 0 or j >= n:
        raise ValueError("Coordinates not within given dimensions.")
    y = (n - 1) // 2 - i
    x = j - (n - 1) // 2
    return x, y


point = (1, 8)
shape = (9, 9, 3)
assert cartesian_to_array(*array_to_cartesian(*point, shape), shape) == point


# Functions to map an image between array and record formats
def image_to_dict(image):
    image = np.atleast_3d(image)
    kv_image = {}
    for i, j in product(range(len(image)), repeat=2):
        kv_image[array_to_cartesian(i, j, image.shape)] = tuple(image[i, j])
    return kv_image


def image_to_df(image):
    return pd.DataFrame(
        [(x, y, r, g, b) for (x, y), (r, g, b) in image_to_dict(image).items()],
        columns=['x', 'y', 'r', 'g', 'b']
    )


def df_to_image(df):
    side = int(len(df) ** 0.5)  # assumes a square image
    return df.set_index(['x', 'y']).to_numpy().reshape(side, side, -1)

In [ ]:
image = df_to_image(df_image)
assert image_to_df(image).equals(df_image)  # ensure transforms are inverses

radius = 128
fig, ax = plt.subplots(figsize=(25, 25))
ax.matshow(image, extent=(-radius, radius+1, -radius, radius+1))
ax.grid(None);

# Naive best neighbour approximation

Now when we have our image as numpy array we can easily estimate lower bound on color transitions total cost as described above.

Code below is one very ineffifient way to find best scoring neighbours for each cell.

In [ ]:
from tqdm.auto import tqdm

size = image.shape[0];
best_neighbor_score = np.zeros((size,size))
for y in tqdm(range(size), total=size):
    for x in range(size):
        best_neighbor_score[x][y] = min([
            np.abs(image[x][y] - image[x+dx][y+dy]).sum() * 3. + np.sqrt(abs(dx) + abs(dy)) - 1
            for dx in range(-8, 9) for dy in range(-8, 9)
            if abs(dx) + abs(dy) <= 8 and abs(dx) + abs(dy) > 0
            and x + dx >= 0 and x + dx < size
            and y + dy >= 0 and y + dy < size
        ])

fig, ax = plt.subplots(figsize=(25, 25))
ax.matshow(best_neighbor_score, extent=(-radius, radius+1, -radius, radius+1))
ax.grid(None);

print('Color score:', np.sum(best_neighbor_score))
print('Solution lower bound:', 66049+np.sum(best_neighbor_score))

# Better lower bound approximation

We know that for each point on the image we need to enter and leave it. So if we choose 2 best scoring neighbours and get their combined score it will give lower bound approximation on twice the total color score.

Code above finds positions of best neighbour for each point to exclude it from second to the best choice.

In [ ]:
best_neighbor_shift = np.zeros((size,size,2), dtype=int)
best_neighbor_pos = np.zeros((size,size,2), dtype=int)
for y in tqdm(range(size), total=size):
    for x in range(size):
        for dx in range(-8, 9):
            for dy in range(-8, 9):
                if abs(dx) + abs(dy) <= 8 and abs(dx) + abs(dy) > 0 \
                        and x + dx >= 0 and x + dx < size and y + dy >= 0 and y + dy < size \
                        and best_neighbor_score[x][y] == np.abs(image[x][y] - image[x+dx][y+dy]).sum() * 3. + np.sqrt(abs(dx) + abs(dy)) - 1:
                    best_neighbor_shift[x][y][:] = [dx,dy]
                    best_neighbor_pos[x][y][:] = [x + dx,y + dy]


Now we can find second to the best neighbours for each cell.

In [ ]:
best_neighbor_of_best_neighbor_score = best_neighbor_score[best_neighbor_pos[:, :, 0], best_neighbor_pos[:, :, 1]]

In [ ]:
second_neighbor_score = np.zeros((size,size))
for y in tqdm(range(size), total=size):
    for x in range(size):
        second_neighbor_score[x][y] = min([
            np.abs(image[x][y] - image[x+dx][y+dy]).sum() * 3. + np.sqrt(abs(dx) + abs(dy)) - 1
            for dx in range(-8, 9) for dy in range(-8, 9)
            if abs(dx) + abs(dy) <= 8 and abs(dx) + abs(dy) > 0
            and x + dx >= 0 and x + dx < size
            and y + dy >= 0 and y + dy < size
            and (dx != best_neighbor_shift[x][y][0] or dy != best_neighbor_shift[x][y][1])
        ])

fig, ax = plt.subplots(figsize=(25, 25))
ax.matshow(second_neighbor_score, extent=(-radius, radius+1, -radius, radius+1))
ax.grid(None);

length_lower_bound = 66049.
color_lower_bound = np.sum(best_neighbor_score+np.minimum(second_neighbor_score,best_neighbor_score+2.*best_neighbor_of_best_neighbor_score+2.))/2.

print('Color score:', np.sum(second_neighbor_score))
print('Solution lower bound:', length_lower_bound+color_lower_bound)

# Why is it interesting?

Looking at second and third to the best neighbours can help determine which points on the picture are more important to optimize for. If all neighbours of a point have same color, we can choose any route without loosing score, however if difference in score between second to the best and third to the best neighbours is big, we better use 2 best neighbours for the route.

So let's keep digging.

In [ ]:
second_neighbor_shift = np.zeros((size,size,2), dtype=int)
for y in tqdm(range(size), total=size):
    for x in range(size):
        for dx in range(-8, 9):
            for dy in range(-8, 9):
                if (dx != best_neighbor_shift[x][y][0] or dy != best_neighbor_shift[x][y][1]) \
                        and abs(dx) + abs(dy) <= 8 and abs(dx) + abs(dy) > 0 \
                        and x + dx >= 0 and x + dx < size and y + dy >= 0 and y + dy < size \
                        and second_neighbor_score[x][y] == np.abs(image[x][y] - image[x+dx][y+dy]).sum() * 3. + np.sqrt(abs(dx) + abs(dy)) - 1:
                    second_neighbor_shift[x][y][:] = [dx,dy]

Now we can find third to best neighbours of each point and visualize costs of choosing these neighbours compared to second best neighbours.

In [ ]:
third_neighbor_score = np.zeros((size,size))
for y in tqdm(range(size), total=size):
    for x in range(size):
        third_neighbor_score[x][y] = min([
            np.abs(image[x][y] - image[x+dx][y+dy]).sum() * 3. + np.sqrt(abs(dx) + abs(dy)) - 1
            for dx in range(-8, 9) for dy in range(-8, 9)
            if abs(dx) + abs(dy) <= 8 and abs(dx) + abs(dy) > 0
            and x + dx >= 0 and x + dx < size
            and y + dy >= 0 and y + dy < size
            and (dx != best_neighbor_shift[x][y][0] or dy != best_neighbor_shift[x][y][1])
            and (dx != second_neighbor_shift[x][y][0] or dy != second_neighbor_shift[x][y][1])
        ])

fig, ax = plt.subplots(figsize=(25, 25))
ax.matshow(third_neighbor_score-second_neighbor_score, extent=(-radius, radius+1, -radius, radius+1))
ax.grid(None);

print('Color score diff:', np.sum(third_neighbor_score-second_neighbor_score))

# Conclusion

As we can see, top leaderboard teams already got relatively close to hypotethic optimal solution, but there is still prenty of room for optimization or even higher lower bound can be found by more involved computations.

It is already can be seen that choosing many suboptimal neighbours may incur pretty significant cost making it impossible to compete for prizes, let alone adding many spare steps to the overall path.

Not surprizingly, paying critical attention to **optimization of areas around cristmas tree branches** will play key role in determining future winners of the competition.

# Wait what about breaking down submission cost?

Yeah let's take sample publicly available submission and see, what contributes more into its total score.

Some modified code from [Getting Started](https://www.kaggle.com/code/ryanholbrook/getting-started-with-santa-2022) Notebook below.

In [ ]:
def get_position(config):
    return reduce(lambda p, q: (p[0] + q[0], p[1] + q[1]), config, (0, 0))

# Total cost of one step: the reconfiguration cost plus the color cost
def step_cost(from_config, to_config, image):
    from_position = cartesian_to_array(*get_position(from_config), image.shape)
    to_position = cartesian_to_array(*get_position(to_config), image.shape)
    distance_cost = np.sqrt(np.abs(np.asarray(from_position) - np.asarray(to_position)).sum())
    rotations_cost = np.sqrt(np.abs(np.asarray(from_config) - np.asarray(to_config)).sum())
    color_cost = np.abs(image[to_position] - image[from_position]).sum() * 3.
    # Count staying at same position separately
    if distance_cost == 0: 
        return [0, rotations_cost, 0, 0, color_cost]
    else:
        return [1, 0, distance_cost - 1, rotations_cost - distance_cost, color_cost]

# Compute total cost of path over image
def total_cost(path, image):
    return reduce(
        lambda cost, pair: cost + step_cost(pair[0], pair[1], image),
        zip(path[:-1], path[1:]),
        np.asarray([0, 0, 0, 0, 0]),
    )

Submitssion and some code from these amazing series of [TSP - Cost Function Start](https://www.kaggle.com/code/asalhi/tsp-cost-function-start-compress-path-split) notebooks

In [ ]:
submission = pd.read_csv('/kaggle/input/santa-2022-sample-submission/submission.csv')
df = submission
df['path'] = df['configuration'].map(lambda x: np.asarray([list(map(int, p2.split(' '))) for p2 in x.split(';')]))
print(len(df))
df.head()

def print_total_cost(path, image):
    costs = total_cost(path, image)
    print('Total cost:\t\t\t\t\t', costs.sum())
    print('\tReconfiguration cost:\t\t\t\t', costs[0] + costs[1] + costs[2] + costs[3])
    print('\t\tReconfiguration baseline:\t\t\t', length_lower_bound)
    print('\t\tDuplicate point visits:\t\t\t\t', costs[0] - length_lower_bound)
    print('\t\tReconfigurations in place:\t\t\t', costs[1])
    print('\t\tAdditional distance cost:\t\t\t', costs[2])
    print('\t\tSpare configuration changes:\t\t\t', costs[3])
    print('\tColor change cost:\t\t\t\t', costs[4])
    print('\t\tColor change baseline:\t\t\t\t', color_lower_bound, '(at least)')
    print('\t\tSuboptimal color transitions:\t\t\t', costs[4] - color_lower_bound, '(at most)')
    
print_total_cost(df['path'], image)

Changing configuration without changing position of an arm tip is one interesting technique that does not add color cost and does not have cost associated with position change, that is why it is counted separately.

Feel free to check your submission stats!